# MultiCamera Paper Results Generation and Plotting

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
from collections import defaultdict
import os
import json
import pandas as pd
import numpy as np
import glob

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.

from utilitysimv4 import Camera, InferenceJob, TrainingJob, MultiPeriodSimulator, fair_naive_scheduler, fair_dumb_scheduler, fair_sco_scheduler, fair_random_scheduler, inference_only_scheduler, single_camera_naive, get_optimus_fn, thief_sco_scheduler, single_camera_optimizer, get_tanh_fn, get_linear_fn, get_infer_profile, get_tanh_fn, simulator, thief_scheduler, fair_scheduler

In [3]:
#PROFILE_DIR = '/home/romilb/research/msr/results/profiling/june_impl/0718_10cityrepl_noretrain_only/meta/profiles'
PROFILE_DIR = '/home/romilb/research/msr/results/profiling_sys/0802_cachebug_fixed/profiles/meta/profiles'
#PROFILE_DIR = '/home/romilb/research/msr/results/profiling_sys/waymo/0726_waymo/meta/profiles'
DATASET='cityscapes' #'cityscapes'
INFERENCE_PROFILE_PATH = '../../real_inference_profiles.csv'
INFERENCE_MAX_DEFAULT = 0.25

## Helper fns

In [4]:
def read_profiles(profile_dir, city_name, use_oracle = False, use_offset = True):
    # use_offset: Enable for waymo.
    # use_oracle: Use oracle data instead of predictions
    profile_path = os.path.join(profile_dir, "{}.json".format(city_name))
    with open(profile_path) as f:
        data = json.load(f)
    if use_offset:
        offset = int(sorted(data["taskwise_train_profiles"], key=lambda x: int(x))[0])
    else:
        offset = 0
    final_data = {"taskwise_train_profiles": {},
                  "taskwise_infer_profiles": {},
                  "taskwise_predicted_profiles": {}}
    for key in data["taskwise_train_profiles"].keys():
        final_data["taskwise_train_profiles"][str(int(key)-offset)] = data["taskwise_train_profiles"][key]
    for key in data["taskwise_infer_profiles"].keys():
        final_data["taskwise_infer_profiles"][str(int(key)-offset)] = data["taskwise_infer_profiles"][key]
    for key in data["taskwise_predicted_profiles"].keys():
        if use_oracle:
            final_data["taskwise_predicted_profiles"][str(int(key)-offset)] = data["taskwise_train_profiles"][key]
        else:
            final_data["taskwise_predicted_profiles"][str(int(key)-offset)] = data["taskwise_predicted_profiles"][key]
    assert "taskwise_train_profiles" in final_data
    assert "taskwise_infer_profiles" in final_data
    return final_data

def create_cameras(city_names, profile_dir = PROFILE_DIR, inference_max_resources = INFERENCE_MAX_DEFAULT, use_oracle = False, use_offset=False):
    cameras = []
    print("Creating cameras for cities {}".format(city_names))
    for idx, city in enumerate(city_names):        
        data_pred = read_profiles(profile_dir, city, use_oracle, use_offset)
        
        TASKS = data_pred["taskwise_train_profiles"].keys()
        # Format = [Acc, res_time, hp_id]
        train_profiles = {TASK_ID: [[x[2], x[3], x[0]] for x in data_pred["taskwise_predicted_profiles"][TASK_ID]] for TASK_ID in TASKS}
        
        # Format = [Acc, res_time, hp_id]
        oracle_profiles = {TASK_ID: [[x[2], x[3], x[0]] for x in data_pred["taskwise_train_profiles"][TASK_ID]] for TASK_ID in TASKS}
        
        # TODO: Picking min profile, which else can we pick?
        infer_start_accs = {TASK_ID: min([acc for hp_id, acc in data_pred["taskwise_infer_profiles"][TASK_ID].items()]) for TASK_ID in TASKS}

        cameras.append(Camera("{}_{}".format(city, idx),
                              train_profiles,
                              oracle_profiles,
                              infer_start_accs,
                              inference_profile_path=INFERENCE_PROFILE_PATH,
                              inference_max_resources=inference_max_resources
                             ))
    return cameras

In [28]:
if DATASET == 'cityscapes':
    test_cities = ['jena', 'cologne', 'zurich']
elif DATASET == 'waymo':
    test_cities = ['phx_20_29', 'phx_30_39', 'phx_40_49']
cameras = create_cameras(test_cities,
                         inference_max_resources = 0.25,
                         use_oracle=True)
print(cameras[0].name)
cameras[0].taskwise_start_accuracy

Creating cameras for cities ['jena', 'cologne', 'zurich']
jena_0


{'1': 0.7894736842105263,
 '2': 0.894736842105263,
 '3': 0.894736842105263,
 '4': 0.8289473684210525,
 '5': 0.8026315789473684,
 '6': 0.8157894736842105,
 '7': 0.8684210526315789,
 '8': 0.6447368421052632,
 '9': 0.8026315789473684}

# Cost vs accuracy - Multi Cam

In [29]:
def get_citywise_data(results, city):
    tasks = list(results.keys())
    tasks.remove('meta')
    final_data = None
    for task in tasks:
        for job in results[task].keys():
            if isinstance(job, InferenceJob):
                if city in job.name:
                    d = results[task][job]
                    time, acc = list(zip(*d))
                    data = pd.DataFrame(acc, columns=['task_{}'.format(task)], index=time)
                    data.index.name = 'time'
                    if final_data is None:
                        final_data = data
                    else:
                        final_data = final_data.join(data)
    return final_data

In [32]:
l=os.listdir(PROFILE_DIR)
cities=[x.split('.')[0] for x in l]
print(cities)

# CITIES for 1,2,3,4,5 varying cities experiment:
# cities = ['jena', 'zurich', 'dusseldorf', 'bremen', 'stuttgart']#, 'strasbourg', 'hanover', 'hamburg', 'krefeld', 'weimar']

#cities for new profiles with cache bug fixed
cities = ['jena', 'zurich', 'darmstadt', 'stuttgart', 'tubingen', 'dusseldorf', 'monchengladbach', 'aachen', 'bochum', 'cologne', 'bremen']

print("Selecting {} cities: {}".format(len(cities), cities))

['cologne', 'zurich', 'darmstadt', 'stuttgart', 'tubingen', 'dusseldorf', 'monchengladbach', 'bremen', 'bochum', 'jena', 'aachen']
Selecting 11 cities: ['jena', 'zurich', 'darmstadt', 'stuttgart', 'tubingen', 'dusseldorf', 'monchengladbach', 'aachen', 'bochum', 'cologne', 'bremen']


In [ ]:
data = []
#cities = ["phx-21-40", "phx-41-60"]
INFER_MAX_RES = 0.25

num_cities = [1,2,5,10]#[len(cities)] #[1,2,3,4,5]
periods_to_test = [100,200] #[10, 50, 100, 200, 400]
provisioned_res_to_test = [1,2,4]#np.arange(0,1,0.2)#,2,4,8,16]#,32,64]#[1,2,4,8,16]#list(np.arange(0,1,0.05))#
task_ids = [str(x) for x in range(1,10)]
use_oracle_modes = [True]#, False]

#camera_names = cities

schedulers_to_test = [(fair_dumb_scheduler, 'fair_dumb'),
                     # (fair_naive_scheduler, 'fair_naive'),
                     # (fair_sco_scheduler, 'fair_sco'),
                     (lambda x,y,z: thief_sco_scheduler(x,y,z,iterations=3,steal_increment=INFER_MAX_RES/3), 'thief'),
                     (inference_only_scheduler, 'inference_only')
                     ]

column_names = ['period', 'resources', 'use_oracle', *[sched_name for _, sched_name in schedulers_to_test]]

for num_cities_to_test in num_cities:
    cities_to_test = cities[0:num_cities_to_test]
    all_data = pd.DataFrame()
    allocation = defaultdict(dict)
    for retraining_period in periods_to_test:
        for provisioned_res in provisioned_res_to_test:
            for use_oracle in use_oracle_modes:
                print(cities_to_test)
                camera_names = cities_to_test
                this_config_result = [retraining_period, provisioned_res, use_oracle]
                for scheduler, sched_name in schedulers_to_test:
                    print("Sched: {}, Resources: {}. Period: {}. Use_Oracle: {}. Num cities: {}".format(
                        sched_name, provisioned_res, retraining_period, use_oracle, num_cities_to_test))
                    cameras = create_cameras(camera_names, inference_max_resources = INFER_MAX_RES, use_oracle=use_oracle)
                    mps = MultiPeriodSimulator(cameras, provisioned_res, scheduler, retraining_period, task_ids = task_ids)
                    results, period_allocation_log = mps.step_till_completion()

                    #Get citywise data
                    for city in cities_to_test:
                        d=get_citywise_data(results, city)
                        d['city']=city
                        d['res']=provisioned_res
                        d['period']=retraining_period
                        d['use_oracle']=use_oracle
                        d['sched'] = sched_name
                        all_data = pd.concat([all_data, d])

                    this_config_result.append(results['meta']['overall_inf_mean_auc'])
                    allocation["{}_{}_{}_{}".format(retraining_period, provisioned_res, sched_name, use_oracle)] = period_allocation_log
                data.append(this_config_result)
                df = pd.DataFrame(data, columns=column_names)
                df.to_csv('data_eval_multicam_cityscapes_histacc_{}cities.csv'.format(num_cities_to_test))
                all_data.to_csv('data_eval_multicam_all_data_cityscapes_histacc_{}cities.csv'.format(num_cities_to_test))

                with open('data_eval_multicam_allocationdata_cityscapes_histacc_{}cities.json'.format(num_cities_to_test), 'w') as outfile:
                    json.dump(allocation, outfile)

50
100
Simulator steps done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0_train_0_17 is done.
Job jena_0

['jena']
Sched: fair_dumb, Resources: 1. Period: 100. Use_Oracle: True. Num cities: 1
Creating cameras for cities ['jena']
Running task 1
Running task 2
Running task 3
WARNING(predictedcfg): Final accuracy 0.8026315789473684 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_8.
WARNING(predictedcfg): Final accuracy 0.7894736842105263 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_17.
WARNING(predictedcfg): Final accuracy 0.763157894736842 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_29.
WARNING(oracle): Final accuracy 0.8026315789473684 less than current acc 0.894736842105263 for camera jena_0, still adding profile if the predictor decides to use it..
WARNING(oracle): Final accuracy 0.7894736842105263 less than current acc 0.894736842105263 for camera jena_0, still adding profile if the predictor decides to use it..
WARNING(oracle): Final accuracy 0.76315789

Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0

Iter 1, final_alloc: {<utilitysimv4.InferenceJob object at 0x7feb5ea02650>: [0.25, 0.75]}
Iter 2, alloc: {<utilitysimv4.InferenceJob object at 0x7feb5ea02650>: [0.25, 0.75]}
Best config for jena_0_inference: jena_0_train_3_8
Iter 2, final_alloc: {<utilitysimv4.InferenceJob object at 0x7feb5ea02650>: [0.25, 0.75]}


Job jena_0_train_3_8 is done.
50
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jena_0_train_3_8 is done.
Job jen

...

Job stuttgart_3_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job darmstadt_2_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job stuttgart_3_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job darmstadt_2_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job stuttgart_3_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job darmstadt_2_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job stuttgart_3_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job darmstadt_2_train_0_8 is done.
100
Job tubingen_4_train_3_8 is done.
Job stuttgart_3_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job darmstadt_2_train_0_8 is done.
Simulator steps done.


Iter 2, final_alloc: {<utilitysimv4.InferenceJob object at 0x7feb5eae4e10>: [0.23333333333333336, 0.0], <utilitysimv4.InferenceJob object at 0x7feb5e892050>: [0.2, 0.31666666666666665], <utilitysimv4.InferenceJob object at 0x7feb5e8038d0>: [0.2, 0.0], <utilitysimv4.InferenceJob object at 0x7feb5ea99710>: [0.2, 0.2], <utilitysimv4.InferenceJob object at 0x7feb5e4b7dd0>: [0.2, 0.45]}
Running task 3
WARNING(predictedcfg): Final accuracy 0.8026315789473684 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_8.
WARNING(predictedcfg): Final accuracy 0.7894736842105263 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_17.
WARNING(predictedcfg): Final accuracy 0.763157894736842 less than current acc 0.894736842105263 for camera jena_0, ignoring profile jena_0_train_1_29.
WARNING(oracle): Final accuracy 0.8026315789473684 less than current acc 0.894736842105263 for camera jena_0, still adding profile if the predicto

Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job jena_0_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job tubingen_4_train_3_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job tubingen_4_train_3

Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job tubingen_4_train_3_8 is done.
Job darmstadt_2_train_0_8 is done.
Job zurich_1_train_3_8 is done.
Job jena_0_train_0_8 is done.
Job stuttgart_3_train_0_8 is done.
Job 

Iter 2, final_alloc: {<utilitysimv4.InferenceJob object at 0x7feb5eae4e10>: [0.2, 0.4], <utilitysimv4.InferenceJob object at 0x7feb5e892050>: [0.2, 0.3166666666666667], <utilitysimv4.InferenceJob object at 0x7feb5e8038d0>: [0.2, 0.0], <utilitysimv4.InferenceJob object at 0x7feb5ea99710>: [0.2, 0.0], <utilitysimv4.InferenceJob object at 0x7feb5e4b7dd0>: [0.2, 0.2833333333333333]}
Running task 4
WARNING(predictedcfg): Final accuracy 0.8841463414634146 less than current acc 0.8902439024390244 for camera stuttgart_3, ignoring profile stuttgart_3_train_1_29.
WARNING(oracle): Final accuracy 0.8841463414634146 less than current acc 0.8902439024390244 for camera stuttgart_3, still adding profile if the predictor decides to use it..
Iter 0, alloc: {<utilitysimv4.InferenceJob object at 0x7feb5eae4e10>: [0.2, 0.2], <utilitysimv4.InferenceJob object at 0x7feb5e892050>: [0.2, 0.2], <utilitysimv4.InferenceJob object at 0x7feb5e8038d0>: [0.2, 0.2], <utilitysimv4.InferenceJob object at 0x7feb5ea99710>

In [28]:
city

'phx_50_59'

In [24]:
print("{}_{}_{}_{}".format(retraining_period, provisioned_res, sched_name, use_oracle))

100_1_fair_dumb_True


In [149]:
x=citywise_data[(citywise_data['sched']=='thief') & (citywise_data['res']==0.8) & (citywise_data['period']==100) & (citywise_data['use_oracle']==True)]# & (citywise_data['city']=="phx-21-40")]

In [150]:
tasks = [t for t in x.mean().keys() if "task" in t]
x.mean()[tasks].mean()

0.8246860780410408

## Profile metadata generation (Run before the simulator, if necessary)

In [5]:
PROFILE_COLUMNS = ["timestamp", "train_time", "train_loss", "train_acc", "train_num_batches",
                            "val_time", "val_loss", "val_acc", "val_num_batches",
                  "test_time", "test_loss", "test_acc", "test_num_batches"]
# profile: [hparamid_epochs, start_acc, end_acc, resource_time]
# def get_data(json_path):
#     with open(json_path) as json_file:
#         data = json.load(json_file)
#     task_ids, accs = zip(*data['test_acc'].items())
#     task_ids = [int(x) for x in task_ids]
#     return task_ids, accs

def get_profile(json_path):
    with open(json_path) as json_file:
        data = json.load(json_file)
    profile_task_map = {}
    for taskid, profile_list in data.items():
        prof = pd.DataFrame(data[taskid], columns=PROFILE_COLUMNS)
        profile_task_map[taskid] = prof
    return profile_task_map

def read_json_file(json_path):
    with open(json_path) as json_file:
        return json.load(json_file)
    
def get_subprofile_epochids(subprofile_json_path):
    data = read_json_file(subprofile_json_path)
    subprofiles_dict = list(data.values())[0]
    return [int(x) for x in subprofiles_dict.keys()]

   
def get_subprofile_default_accs(subprofile_json_path):
    data = read_json_file(subprofile_json_path)
    first_task = sorted(data.keys(), key = lambda x: int(x))[0]
    return list(data[first_task].values())[0] # This is taskwise accuracy of untrained model

# def get_top5(taskwise_acc_dict, taskid, topk=5):
#     data = taskwise_acc_dict[taskid]
#     hp_ids = sorted(data, key=data.get, reverse=True)[:topk]
#     return hp_ids, [data[i] for i in hp_ids]

In [6]:
# Use this function to generate subprofiles from profiling output.
# If use_predictions is false, writes oracle info as predictions.

def generate_meta_profiles(results_root, use_predictions = True, epochs_to_subprofile=[8,17,29]):
    result_dir = os.path.join(results_root, "meta")
    
    profiles_dir = os.path.join(result_dir, "profiles")
    os.makedirs(profiles_dir, exist_ok=True)
    city_names = [d for d in os.listdir(results_root) if os.path.isdir(os.path.join(results_root, d))]
    if "meta" in city_names:
        city_names.remove("meta")
    if "plots" in city_names:
        city_names.remove("plots")
    print(city_names)
    
    for city in city_names:
        city_path = os.path.join(results_root, city)

        configs = {}

        with open(os.path.join(city_path, 'hyp_map.json')) as f:
            hyp_map = json.load(f)
        profile_files = (glob.glob(city_path + "/*_profile.json"))
        hyperparam_ids = [os.path.basename(x).split('_profile.json')[0] for x in profile_files]

        baseline_taskwise_acc = defaultdict(dict)
        train_taskwise_profiles = defaultdict(list)
        predicted_taskwise_profiles = defaultdict(list)
        
        print(hyperparam_ids)
        for hp_id in hyperparam_ids:
            profile_path = os.path.join(city_path, '{}_profile.json'.format(hp_id))
            subprofile_path = os.path.join(city_path, '{}_subprofile_test.json'.format(hp_id))
            if use_predictions:
                predictions_path = os.path.join(city_path, '{}_predicted_acc_1.json'.format(hp_id))
                predicted_profiles = read_json_file(predictions_path)
            taskwise_profiles = get_profile(profile_path)
            
            # For getting pre-retraining accuracy:
            retraining_result_path = os.path.join(city_path, '{}_retraining_result.json'.format(hp_id))
            retraining_result = read_json_file(retraining_result_path)
            
            #epochs_to_subprofile = get_subprofile_epochids(subprofile_path) # [epoch ids where subprofiled]
            #untrained_accs = get_subprofile_default_accs(subprofile_path) # {task_id: acc}
            TASK_IDS = sorted(taskwise_profiles.keys(), key = lambda x: int(x))
            TASK_IDS = [t for t in TASK_IDS if len(taskwise_profiles[t]) > 0]
            
            
            print(epochs_to_subprofile)
            print(TASK_IDS)
            
            for TASK_ID in [str(x) for x in TASK_IDS]:
                profile = taskwise_profiles[TASK_ID]
                if use_predictions:
                    prediction = predicted_profiles[str(int(TASK_ID)-1)]

                # Get baseline no retraining inference accuracies
                task_start_acc = retraining_result["preretrain_test_acc"][TASK_ID]
                baseline_taskwise_acc[TASK_ID][hp_id] = task_start_acc
                for e in epochs_to_subprofile:
                    resource_time = sum(profile.loc[0:e]['train_time'])
                    start_acc = task_start_acc
                    actual_end_acc = profile.loc[e]['test_acc']
                    if use_predictions:
                        predicted_end_acc = prediction[str(e)][TASK_ID] # ERROR - +1 wont exist always
                    else:
                        predicted_end_acc = actual_end_acc
                    name = "{}_{}".format(hp_id, e)
                    train_taskwise_profiles[TASK_ID].append([name, start_acc, actual_end_acc, resource_time])
                    predicted_taskwise_profiles[TASK_ID].append([name, start_acc, predicted_end_acc, resource_time])
                    #TODO: ROMILB: Calculate end_acc by looking at task:task+1. Time is from task
        infer_train_profile = {'taskwise_train_profiles': train_taskwise_profiles,
                               'taskwise_infer_profiles':baseline_taskwise_acc,
                               'taskwise_predicted_profiles': predicted_taskwise_profiles}
        with open(os.path.join(profiles_dir, "{}.json".format(city, TASK_ID)), 'w') as outfile:
            json.dump(infer_train_profile, outfile)
    return infer_train_profile

In [7]:
results_root = '/home/romilb/research/msr/results/profiling_sys/0802_cachebug_fixed/profiles/'
#results_root = '/home/romilb/research/msr/results/profiling/june_impl/0718_10cityrepl_noretrain_only/'

p = generate_meta_profiles(results_root, use_predictions=False, epochs_to_subprofile=[8,17,29])

['zurich', 'stuttgart', 'darmstadt', 'dusseldorf', 'monchengladbach', 'aachen', 'tubingen', 'bochum', 'bremen', 'cologne', 'jena']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2',

In [13]:
p['taskwise_train_profiles']['4']

[['2_8', 0.8289473684210525, 0.9736842105263157, 48.228275775909424],
 ['2_17', 0.8289473684210525, 0.9736842105263157, 96.34552574157715],
 ['2_29', 0.8289473684210525, 0.9736842105263157, 160.5095088481903],
 ['5_8', 0.8289473684210525, 0.9868421052631579, 112.08905005455017],
 ['5_17', 0.8289473684210525, 1.0, 224.4740161895752],
 ['5_29', 0.8289473684210525, 0.9868421052631579, 374.28492641448975],
 ['3_8', 0.8289473684210525, 0.9736842105263157, 11.10056209564209],
 ['3_17', 0.8289473684210525, 0.9605263157894737, 22.262295961380005],
 ['3_29', 0.8289473684210525, 0.9605263157894737, 37.267428398132324],
 ['0_8', 0.8289473684210525, 0.9342105263157894, 4.810425281524658],
 ['0_17', 0.8289473684210525, 0.9078947368421052, 9.619254350662231],
 ['0_29', 0.8289473684210525, 0.9078947368421052, 16.01638174057007],
 ['1_8', 0.8289473684210525, 0.9605263157894737, 24.189301013946533],
 ['1_17', 0.8289473684210525, 0.9473684210526315, 48.28698992729187],
 ['1_29', 0.8289473684210525, 0.93

In [24]:
results_root = '/home/romilb/research/msr/results/profiling/june_impl/20200617_2121_no101_no50/'
p = generate_meta_profiles(results_root, use_predictions=False, epochs_to_subprofile=[8,17,29])

['zurich', 'cologne', 'jena']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6',

In [29]:
results_root = '/home/romilb/research/msr/results/profiling_sys/waymo/0726_waymo/'
p = generate_meta_profiles(results_root, use_predictions=False, epochs_to_subprofile=[8,17,29])

['phx3_30_39', 'phx2_50_59', 'phx2_40_49', 'phx_20_29', 'phx_40_49', 'phx_30_39', 'phx2_20_29', 'phx_50_59', 'phx2_30_39', 'phx3_20_29']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
['2', '5', '3', '0', '1', '4']
[8, 17, 29]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
[8, 17, 29]
['1'

In [17]:
p['taskwise_infer_profiles'].keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9'])

# For legacy only profiles

In [26]:
# Use this function to generate subprofiles from profiling output.
# If use_predictions is false, writes oracle info as predictions.

def generate_meta_profiles(results_root, use_predictions = True, epochs_to_subprofile=[8,17,29]):
    result_dir = os.path.join(results_root, "meta")
    
    profiles_dir = os.path.join(result_dir, "profiles")
    os.makedirs(profiles_dir, exist_ok=True)
    city_names = [d for d in os.listdir(results_root) if os.path.isdir(os.path.join(results_root, d))]
    if "meta" in city_names:
        city_names.remove("meta")
    if "plots" in city_names:
        city_names.remove("plots")
    print(city_names)
    
    for city in city_names:
        city_path = os.path.join(results_root, city)

        configs = {}

        with open(os.path.join(city_path, 'hyp_map.json')) as f:
            hyp_map = json.load(f)
        profile_files = (glob.glob(city_path + "/*_profile.json"))
        hyperparam_ids = [os.path.basename(x).split('_profile.json')[0] for x in profile_files]

        baseline_taskwise_acc = defaultdict(dict)
        train_taskwise_profiles = defaultdict(list)
        predicted_taskwise_profiles = defaultdict(list)
        
        print(hyperparam_ids)
        for hp_id in hyperparam_ids:
            profile_path = os.path.join(city_path, '{}_profile.json'.format(hp_id))
            subprofile_path = os.path.join(city_path, '{}_subprofile_test.json'.format(hp_id))
            if use_predictions:
                predictions_path = os.path.join(city_path, '{}_predicted_acc_1.json'.format(hp_id))
                predicted_profiles = read_json_file(predictions_path)
            taskwise_profiles = get_profile(profile_path)
            
            # For getting pre-retraining accuracy:
            retraining_result_path = os.path.join(city_path, '{}_retraining_result.json'.format(hp_id))
            retraining_result = read_json_file(retraining_result_path)
            
            #epochs_to_subprofile = get_subprofile_epochids(subprofile_path) # [epoch ids where subprofiled]
            #untrained_accs = get_subprofile_default_accs(subprofile_path) # {task_id: acc}
            TASK_IDS = sorted(taskwise_profiles.keys(), key = lambda x: int(x))
            TASK_IDS = [t for t in TASK_IDS if len(taskwise_profiles[t]) > 0]
            
            
            print(epochs_to_subprofile)
            print(TASK_IDS)
            
            for TASK_ID in [str(x) for x in TASK_IDS]:
                profile = taskwise_profiles[TASK_ID]
                if use_predictions:
                    prediction = predicted_profiles[str(int(TASK_ID)-1)]

                # Get baseline no retraining inference accuracies
                task_start_acc = profile.loc[0]['test_acc']
                baseline_taskwise_acc[TASK_ID][hp_id] = task_start_acc
                for e in epochs_to_subprofile:
                    resource_time = sum(profile.loc[0:e]['train_time'])
                    start_acc = task_start_acc
                    actual_end_acc = profile.loc[e]['test_acc']
                    if use_predictions:
                        predicted_end_acc = prediction[str(e)][TASK_ID] # ERROR - +1 wont exist always
                    else:
                        predicted_end_acc = actual_end_acc
                    name = "{}_{}".format(hp_id, e)
                    train_taskwise_profiles[TASK_ID].append([name, start_acc, actual_end_acc, resource_time])
                    predicted_taskwise_profiles[TASK_ID].append([name, start_acc, predicted_end_acc, resource_time])
                    #TODO: ROMILB: Calculate end_acc by looking at task:task+1. Time is from task
        infer_train_profile = {'taskwise_train_profiles': train_taskwise_profiles,
                               'taskwise_infer_profiles':baseline_taskwise_acc,
                               'taskwise_predicted_profiles': predicted_taskwise_profiles}
        with open(os.path.join(profiles_dir, "{}.json".format(city, TASK_ID)), 'w') as outfile:
            json.dump(infer_train_profile, outfile)
    return infer_train_profile

In [27]:
results_root = '/home/romilb/research/msr/results/profiling_sys/0716_fixedseed_10cam/profiles/'
#results_root = '/home/romilb/research/msr/results/profiling/june_impl/0718_10cityrepl_noretrain_only/'

p = generate_meta_profiles(results_root, use_predictions=False, epochs_to_subprofile=[0])

['zurich', 'stuttgart', 'darmstadt', 'dusseldorf', 'monchengladbach', 'aachen', 'tubingen', 'bochum', 'bremen', 'cologne', 'jena']
['2', '5', '3', '0', '1', '4']


NameError: name 'get_profile' is not defined